# TensorRec recommender engine:
## Prototyping of a recommender system in Python using TensorRec including input data manipulation, algorithm design, and usage for prediction.

#### TensorRec is a Python package for building recommender systems. A TensorRec recommender system consumes three pieces of input data: user features, item features, and interactions. Based on the user/item features, the system will predict which items to recommend. The interactions are used when fitting the model: predictions are compared to the interactions and a loss/penalty is calculated, which the system learns to decrease. As we prototype our system, we tackle three major situations: how we handle interactions, how we handle features, and how we structure the recommender itself.


<img src="https://miro.medium.com/max/1400/1*YotDpHjvGL8xK91ZggthbA.png" />


https://towardsdatascience.com/getting-started-with-recommender-systems-and-tensorrec-8f50a9943eef

### Raw ratings load :Each row represents a single rating: one user and one item. We’ll be using these ratings(frequency of purchase of each item) as our interactions between the user and the product.

In [2]:
import csv
# Open and read in the ratings file
print('Loading ratings')
with open('/user-home/libraries/bb_library/datasets/dec16nov17_cf.csv', 'r') as ratings_file:
    ratings_file_reader = csv.reader(ratings_file)
    raw_ratings = list(ratings_file_reader)
    raw_ratings_header = raw_ratings.pop(0)

raw_ratings

Loading ratings


[['25146', '1104015', '1', '2017-11'],
 ['25146', '1105001', '1', '2017-11'],
 ['25146', '1105007', '1', '2017-11'],
 ['25146', '1105068', '2', '2017-11'],
 ['25146', '1106234', '1', '2017-11'],
 ['25146', '1313152', '1', '2017-11'],
 ['25146', '1701305', '1', '2017-11'],
 ['25146', '2229008', '1', '2017-11'],
 ['25146', '2248001', '1', '2017-11'],
 ['88418', '1103087', '2', '2017-11'],
 ['88418', '1306189', '1', '2017-11'],
 ['88418', '1310080', '1', '2017-11'],
 ['88418', '1310121', '2', '2017-11'],
 ['88418', '1314063', '2', '2017-11'],
 ['88418', '1316005', '1', '2017-11'],
 ['88418', '1318009', '1', '2017-11'],
 ['88418', '1318073', '1', '2017-11'],
 ['88418', '1318203', '1', '2017-11'],
 ['88418', '1320143', '1', '2017-11'],
 ['88418', '1380073', '1', '2017-11'],
 ['88418', '1387299', '1', '2017-11'],
 ['88418', '1701305', '1', '2017-11'],
 ['88418', '1860241', '1', '2017-11'],
 ['88418', '1860308', '2', '2017-11'],
 ['88418', '1862008', '2', '2017-11'],
 ['88418', '2172003', '1'

### Iterate through the input to map Item and User IDs to new internal IDs
### The new internal IDs will be created by the defaultdict on insertion

In [3]:
import collections

idcte_to_internal_user_ids = collections.defaultdict(lambda: len(idcte_to_internal_user_ids))
idfam1_to_internal_item_ids = collections.defaultdict(lambda: len(idfam1_to_internal_item_ids))
for row in raw_ratings:
    row[0] = idcte_to_internal_user_ids[int(row[0])]
    row[1] = idfam1_to_internal_item_ids[int(row[1])]
    row[2] = float(row[2])
n_users = len(idcte_to_internal_user_ids)
n_items = len(idfam1_to_internal_item_ids)

In [54]:
idfam1_to_internal_item_ids
#idcte_to_internal_user_ids

defaultdict(<function __main__.<lambda>>,
            {37748737: 431561,
             37748738: 431562,
             37748739: 431563,
             37748741: 431564,
             37748742: 431565,
             37748743: 431566,
             37748744: 431567,
             37748745: 431568,
             37748748: 431569,
             37748749: 431570,
             37748750: 431571,
             37748751: 431572,
             37748753: 431573,
             37748754: 431574,
             37748759: 431575,
             37748760: 431576,
             37748762: 431577,
             37748764: 431578,
             37748767: 431579,
             37748770: 431580,
             37748771: 431581,
             37748773: 431582,
             37748775: 431583,
             37748777: 431584,
             37748779: 431585,
             37748780: 431586,
             37748781: 431587,
             37748783: 431588,
             37748786: 431589,
             37748791: 431590,
             37748793: 43159

In [5]:
from collections import defaultdict
import csv
import numpy 
import random
from scipy import sparse
from sklearn.preprocessing import MultiLabelBinarizer

### At this point, we’ll break the ratings in to a training and test set by shuffling and splitting the ratings. Our prototypes will be trained on the training set, and we’ll evaluate their success using the test set. Splitting the train/test sets at random like this is crude, and there are more rigorous techniques for model evaluation, but it is quick and clear for the purposes of this example. 

In [6]:
# Shuffle the ratings and split them in to train/test sets 80%/20%
random.shuffle(raw_ratings)  # Shuffles the list in-place
cutoff = int(.8 * len(raw_ratings))
train_ratings = raw_ratings[:cutoff]
test_ratings = raw_ratings[cutoff:]


### Next, we reorganize these ratings in to a Scipy sparse matrix. In this matrix, every row represents a user and every column is an item. The [i, j]th value in this matrix is User i’s interaction with Item j.

In [7]:

# This method converts a list of (user, item, rating) to a sparse matrix
def interactions_list_to_sparse_matrix(interactions):
    users_column, items_column, ratings_column, _= zip(*interactions)
    return sparse.coo_matrix((ratings_column, (users_column, items_column)),
                             shape=(n_users, n_items))


# Create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(raw_ratings)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)



In [8]:
sparse_train_ratings

<591000x1983 sparse matrix of type '<class 'numpy.float64'>'
	with 4804957 stored elements in COOrdinate format>

### TensorRec library runs on TensorFlow so we install a compatible version of TensorFlow 
### Both TensorFlow and TensorRec can be installed using !pip

In [9]:
!pip install "tensorflow==1.7.0"

  Using cached https://files.pythonhosted.org/packages/e9/f2/85074ef76215435df20424d844ec816428750add61109865706a6bf808c6/tensorflow-1.7.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e5/3a/3df7d5a06d65f8d3064bb03b0e11e3092d125f933eb11ab658a4d589b92c/grpcio-1.23.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/0b/ec/65d4e8410038ca2a78c09034094403d231228d0ddcae7d470b223456e55d/tensorboard-1.7.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Found existing installation: tensorflow 1.4.1
    Uninstalling tensorflow-1.4.1:
      Successfully uninstalled tensorflow-1.4.1
You are using pip version 19.0.3, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import tensorflow as tf
print(tf.__version__)

1.7.0


In [11]:
!pip install tensorrec --ignore-installed

  Using cached https://files.pythonhosted.org/packages/1d/14/608abc1c669caf7f2f9a7a13d237bd3f58df08ed71b8eb4c2d756b025adf/tensorrec-0.26.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7a/0e/3781e028d62a8422244582abd8f084e6314297026760587c85607f687bf3/scipy-1.3.1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/64/7619774f0bd8ef364d46a5df8eb1bc78784cd787324b9624f6793e72f787/numpy-1.17.1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7c/fb/7b2c5b3e85ad335b53ca67deb2ef4af574dc0a8759f43b7f45e15005e449/tensorflow-1.14.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cache

In [12]:
import tensorrec

## Collaborative Filter Prototype
### A collaborative filter is an algorithm that learns which users have similar tastes and recommends items to a user based on what similar users have liked. A common way to do this is through matrix factorization. In matrix factorization, we have to learn two matrices (user representations and item representations) that, when multiplied together, approximate the interactions:
#### TensorRec will perform matrix factorization by default if it is given only identity matrices as user/item features. These identity matrices are often called “indicator features.”

In [13]:
# Construct indicator features for users and items
user_indicator_features = sparse.identity(n_users)
item_indicator_features = sparse.identity(n_items)

# Build a matrix factorization collaborative filter model
cf_model = tensorrec.TensorRec(n_components=5)

# Fit the collaborative filter model
print("Training collaborative filter")
cf_model.fit(interactions=sparse_train_ratings,
             user_features=user_indicator_features,
             item_features=item_indicator_features)

Training collaborative filter


/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### Performance of the model: first prototype
#### To do this, we’ll look at a metric called “recall at K.” Recall@K says, for the average user, what percentage of their test items made it in to the top K in the predicted rankings.
#### Recall@K is a nice metric for many recommender systems because it emulates the behavior of a recommendation product. Before calculating the recall, we’ll want to decide which interactions should count as a “purchased item.” In this case, choose to use all ratings of at least 1.0 as “liked products” and ignore the rest. 

In [ ]:
# Create sets of train/test interactions that are only frequency > 1 since these represent the products that have been purchased 
sparse_train_ratings_1plus = sparse_train_ratings.multiply(sparse_train_ratings >= 1)
sparse_test_ratings_1plus = sparse_test_ratings.multiply(sparse_test_ratings >= 1)


# This method consumes item ranks for each user and prints out train/test metrics
def check_results(ranks):
    train_recall_at_10 = tensorrec.eval.recall_at_k(
        test_interactions=sparse_train_ratings_1plus,
        predicted_ranks=ranks,
        k=10
    ).mean()
    test_recall_at_10 = tensorrec.eval.recall_at_k(
        test_interactions=sparse_test_ratings_1plus,
        predicted_ranks=ranks,
        k=10
    ).mean()
    print("Performance metrics: Train: {:.4f} Test: {:.4f}".format(train_recall_at_10,
                                                            test_recall_at_10))


# Check the results of the MF CF model
print("Matrix factorization collaborative filter:")
predicted_ranks = cf_model.predict_rank(user_features=user_indicator_features,
                                        item_features=item_indicator_features)
check_results(predicted_ranks)

## Loss Graphs
### One way we can configure our TensorRec system is by changing the loss graph. The loss graph takes in predictions and interactions and calculates a penalty (loss) that the system will try to decrease as it learns.
#### WMRB, which stands for “weighted margin-rank batch,” works by taking a random sample of items the user hasn’t interacted with and comparing their predictions to items the user likes. Over time, this pushes items a user likes to the top of the rankings. We can try using different loss graphs like WARP

In [ ]:
# Let's try a new loss function: WMRB 
print("Training collaborative filter with WMRB loss")
ranking_cf_model = tensorrec.TensorRec(n_components=5,
                                       loss_graph=tensorrec.loss_graphs.WMRBLossGraph())
ranking_cf_model.fit(interactions=sparse_train_ratings_1plus,
                     user_features=user_indicator_features,
                     item_features=item_indicator_features,
                     n_sampled_items=int(n_items *1))

# Check the results of the WMRB MF CF model
print("WMRB matrix factorization collaborative filter:")
predicted_ranks = ranking_cf_model.predict_rank(user_features=user_indicator_features,
                                                item_features=item_indicator_features)

# Adding Metadata Features
## To continue experimenting, we should try to make use of other data available to us. We will try using User Demographic data

In [24]:
import csv
# To improve the recommendations, lets read in the user demographic data
print('Loading user metadata')
with open('/user-home/libraries/Sampled_data/datasets/dec16nov17_featuresCombined.csv', 'r') as users_file:
    users_file_reader = csv.reader(users_file)
    raw_user_metadata = list(users_file_reader)
    raw_user_metadata_header = raw_user_metadata.pop(0)
raw_user_metadata

Loading user metadata


[['25146', '77,C,M', 'PUEBLA                   '],
 ['88418', '64,C,F', 'PUEBLA                   '],
 ['98505', '89,C,F', 'PUEBLA                   '],
 ['132310', '59,C,M', 'PUEBLA                   '],
 ['160543', '67,C,M', 'PUEBLA                   '],
 ['162049', '57,C,M', 'PUEBLA                   '],
 ['189052', '71,C,F', 'PUEBLA                   '],
 ['194526', '58,C,M', 'PUEBLA                   '],
 ['194755', '74,S,F', 'PUEBLA                   '],
 ['203252', '51,C,M', 'PUEBLA                   '],
 ['212877', '55,D,F', 'PUEBLA                   '],
 ['268770', '54,C,F', 'PUEBLA                   '],
 ['332960', '74,C,M', 'PUEBLA                   '],
 ['336514', '76,S,F', 'PUEBLA                   '],
 ['384068', '53,C,M', 'PUEBLA                   '],
 ['385281', '74,S,M', 'PUEBLA                   '],
 ['386884', '45,S,M', 'PUEBLA                   '],
 ['415494', '55,C,F', 'PUEBLA                   '],
 ['440759', '64,S,F', 'PUEBLA                   '],
 ['535790', '53

### First, we’ll want to read this data, map the movies to our internal IDs, and keep track of the features for each user. Then we’ll binarize the feature  labels using Scikit’s MultiLabelBinarizer. The binarized output will be our features for our new recommender system.

In [25]:
# Map the features IDs to our internal IDs and keep track of the gender and age
user_id_by_internal_id = {}
user_features_by_internal_id = {}
for row in raw_user_metadata:
    row[0] = idfam1_to_internal_item_ids[int(row[0])]  # Map to IDs
    row[1] = row[1].split(',')  # Split up
    user_id_by_internal_id[row[0]] = row[0]
    user_features_by_internal_id[row[0]] = row[1]

# Look at an example user metadata row
print("Raw metadata example:\n{}\n{}".format(raw_user_metadata_header, 
                                             raw_user_metadata[0]))



Raw metadata example:
['USERID', 'FEATURES', 'STATE']
[1983, ['77', 'C', 'M'], 'PUEBLA                   ']


In [26]:
user_features_by_internal_id

{1983: ['77', 'C', 'M'],
 1984: ['64', 'C', 'F'],
 1985: ['89', 'C', 'F'],
 1986: ['59', 'C', 'M'],
 1987: ['67', 'C', 'M'],
 1988: ['57', 'C', 'M'],
 1989: ['71', 'C', 'F'],
 1990: ['58', 'C', 'M'],
 1991: ['74', 'S', 'F'],
 1992: ['51', 'C', 'M'],
 1993: ['55', 'D', 'F'],
 1994: ['54', 'C', 'F'],
 1995: ['74', 'C', 'M'],
 1996: ['76', 'S', 'F'],
 1997: ['53', 'C', 'M'],
 1998: ['74', 'S', 'M'],
 1999: ['45', 'S', 'M'],
 2000: ['55', 'C', 'F'],
 2001: ['64', 'S', 'F'],
 2002: ['53', 'C', 'M'],
 2003: ['65', 'C', 'M'],
 2004: ['56', 'C', 'F'],
 2005: ['50', 'S', 'F'],
 2006: ['52', 'C', 'F'],
 2007: ['49', 'C', 'M'],
 2008: ['61', 'C', 'M'],
 2009: ['59', 'S', 'F'],
 2010: ['49', 'C', 'F'],
 2011: ['49', 'C', 'F'],
 2012: ['79', 'V', 'F'],
 2013: ['65', 'S', 'M'],
 2014: ['59', 'C', 'M'],
 2015: ['57', 'C', 'F'],
 2016: ['53', 'C', 'F'],
 2017: ['56', 'C', 'F'],
 2018: ['49', 'C', 'F'],
 2019: ['53', 'C', 'F'],
 2020: ['59', 'C', 'F'],
 2021: ['46', 'C', 'F'],
 2022: ['49', 'S', 'F'],


###  Build a list of features where the index is the internal user ID and the value is a list of features

In [27]:
user_feat = [user_features_by_internal_id[internal_id]
                for internal_id in user_features_by_internal_id]

In [28]:
user_feat

[['77', 'C', 'M'],
 ['64', 'C', 'F'],
 ['89', 'C', 'F'],
 ['59', 'C', 'M'],
 ['67', 'C', 'M'],
 ['57', 'C', 'M'],
 ['71', 'C', 'F'],
 ['58', 'C', 'M'],
 ['74', 'S', 'F'],
 ['51', 'C', 'M'],
 ['55', 'D', 'F'],
 ['54', 'C', 'F'],
 ['74', 'C', 'M'],
 ['76', 'S', 'F'],
 ['53', 'C', 'M'],
 ['74', 'S', 'M'],
 ['45', 'S', 'M'],
 ['55', 'C', 'F'],
 ['64', 'S', 'F'],
 ['53', 'C', 'M'],
 ['65', 'C', 'M'],
 ['56', 'C', 'F'],
 ['50', 'S', 'F'],
 ['52', 'C', 'F'],
 ['49', 'C', 'M'],
 ['61', 'C', 'M'],
 ['59', 'S', 'F'],
 ['49', 'C', 'F'],
 ['49', 'C', 'F'],
 ['79', 'V', 'F'],
 ['65', 'S', 'M'],
 ['59', 'C', 'M'],
 ['57', 'C', 'F'],
 ['53', 'C', 'F'],
 ['56', 'C', 'F'],
 ['49', 'C', 'F'],
 ['53', 'C', 'F'],
 ['59', 'C', 'F'],
 ['46', 'C', 'F'],
 ['49', 'S', 'F'],
 ['46', 'C', 'M'],
 ['58', 'S', 'F'],
 ['50', 'C', 'M'],
 ['47', 'C', 'M'],
 ['49', 'U', 'F'],
 ['59', 'S', 'F'],
 ['44', 'C', 'F'],
 ['49', 'C', 'M'],
 ['58', 'C', 'M'],
 ['49', 'U', 'M'],
 ['53', 'C', 'M'],
 ['74', 'C', 'F'],
 ['48', 'C',

In [30]:
# Transform the features into binarized labels using scikit's MultiLabelBinarizer
user_features = MultiLabelBinarizer().fit_transform(user_feat)
n_features = user_features.shape[1]

In [31]:
user_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

### Coerce the user features to a sparse matrix, which TensorRec expects


In [33]:
user_features_mat = sparse.coo_matrix(user_features)
user_features_mat

<591000x86 sparse matrix of type '<class 'numpy.int64'>'
	with 1772998 stored elements in COOrdinate format>

## Content-based Recommendation
### Now that we have metadata about our user, one thing we can try is to recommend based solely on the user metadata.
### To do this, we will configure a TensorRec model to use a pass-through representation graph for item features. For us, this means that the user representations will be the same as the user features that are passed in (just the user information like gender, age etc.) and the item representations will reflect how much the item suits that particular set of user features.
#### Ideal case is when we would have item metadata as well: because that would have a greater impact on making the recommendation better- also help solving the cold start problem. There is a major weakness to this system: these features alone are not very descriptive and are not enough information to make an informed recommendation.


In [2]:
# Fit a content-based model using the user features
print("Training content-based recommender")
content_model = tensorrec.TensorRec(
    n_components=n_features,
   user_repr_graph=tensorrec.representation_graphs.FeaturePassThroughRepresentationGraph()
    
)

Training content-based recommender


In [45]:
content_model.fit(interactions=sparse_train_ratings_1plus,
                  user_features=user_features_mat,
                  item_features=item_indicator_features,
                  n_sampled_items=int(n_items * .01))

/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [46]:
# Check the results of the content-based model
print("Content-based recommender:")
predicted_ranks = content_model.predict_rank(user_features=user_features_mat,
                                             item_features=item_indicator_features)
check_results(predicted_ranks)

Content-based recommender:
Recall at 10: Train: 0.0029 Test: 0.0031


### It’s not as good as the ranking collaborative filter but it’s significantly more effective if we add it to the base collaborative filter.
#### There is a major weakness to this system: user feature alone are not very descriptive and are not enough information to make an informed recommendation. If we had more descriptive metadata and item metadata (views, clicks, basket information etc.) we may have more success with this content-based recommender system.
#### On the other hand, there is a major strength to this system: by relying on only metadata features, and not using indicator features, we can recommend products which were not present when training the model. Similarly, if we have valuable user metadata we can avoid using user indicator features and make predictions for users who’ve never interacted with a product before. This is called “cold-start” recommendation.

# Hybrid recommender: 
## Hybrid recommender systems combine two or more recommendation strategies in different ways to benefit from their complementary advantages.
### Let’s combine these two: we’ll use indicator features to get the strengths of a collaborative filter, and we’ll also use the content features to take advantage of the metadata. This combination of collaborative filtering and content-based recommendation is the hybrid model.


#### We do this by stacking the two sets of features together:

In [34]:
# Try concatenating the user features on to the indicator features for a hybrid recommender system
full_user_features = sparse.hstack([user_indicator_features, user_features_mat])
full_user_features

<591000x591086 sparse matrix of type '<class 'numpy.float64'>'
	with 2363998 stored elements in COOrdinate format>

In [ ]:
print("Training hybrid recommender")
hybrid_model = tensorrec.TensorRec(
    n_components=5
)
hybrid_model.fit(interactions=sparse_train_ratings_1plus,
                 user_features=full_user_features,
                 item_features=item_indicator_features,
                 n_sampled_items=int(n_items * .01))


### This performs the best even though we are using trivial features from users. If we have more metadata, we can expect larger impact of the Hybrid recommender

In [36]:
print("Hybrid recommender:")
predicted_ranks = hybrid_model.predict_rank(user_features=full_user_features,
                                            item_features=item_indicator_features)
check_results(predicted_ranks)

Hybrid recommender:
Recall at 10: Train: 0.2035 Test: 0.1995


## Making recommendations
### We do this by passing the user’s feature vector and all the item features to predict_rank() and examining the resulting rankings

In [50]:
# Pull user features out of the user features matrix and predict for just that user
u_features = sparse.csr_matrix(user_indicator_features)[2001]
u_rankings = hybrid_model.predict_rank(user_features=u432_features,
                                          item_features=item_indicator_features)[0]

# Get internal IDs of User 432's top 10 recommendations
# These are sorted by item ID, not by rank
# This may contain items with which User 432 has already interacted
u_top_ten_recs = numpy.where(u432_rankings <= 10)[0]
print("User x: Item recommendations:")
u_top_ten_recs

User x: Item recommendations:


array([  68,  423,  454,  616, 1133, 1452, 1505, 1569, 1596, 1916])

#### The value of the range over which the recommender should iterate has to be the same as the # of the users

In [ ]:
# Pull user features out of the user features matrix and predict for just all users
for user in range(n_users):
    u_features = sparse.csr_matrix(full_user_features)[user]
    u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)[0]
    u_top_ten_recs = numpy.where(u_rankings <= 10)[0]
    print("User"+str(user)+": Item recommendations:")
    print(u_top_ten_recs)


#### Converting Internal IDs back to the original IDs

In [ ]:
for user in range(10):
    u_features = sparse.csr_matrix(full_user_features)[user]
    u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)[0]
    u_top_ten_recs = numpy.where(u_rankings <= 10)[0]
    print("User "+str(list(idcte_to_internal_user_ids.keys())[list(idcte_to_internal_user_ids.values()).index(user)])+": Item recommendations:")
    #print(list(idcte_to_internal_user_ids.keys())[list(idcte_to_internal_user_ids.values()).index(user)]) 
    for m in u_top_ten_recs:
        print(list(idfam1_to_internal_item_ids.keys())[list(idfam1_to_internal_item_ids.values()).index(m)]) 



## Creating and writing a resulting CSV for recommendations for all users in the input database

In [ ]:
import pandas as pd

HybridRecommendations=pd.DataFrame([])

In [ ]:
for user in range(n_users):
    u_features = sparse.csr_matrix(full_user_features)[user]
    u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)[0]
    u_top_ten_recs = numpy.where(u_rankings <= 10)[0]
    user_id =str(list(idcte_to_internal_user_ids.keys())[list(idcte_to_internal_user_ids.values()).index(user)])
    for m in u_top_ten_recs:
        items = (list(idfam1_to_internal_item_ids.keys())[list(idfam1_to_internal_item_ids.values()).index(m)]) 
        HybridRecommendations=HybridRecommendations.append(pd.DataFrame({'itemId': items,'userId': user_id}, index=[0]), ignore_index=True)

In [ ]:
HybridRecommendations=HybridRecommendations[['userId','itemId']]
HybridRecommendations.head()

In [ ]:
HybridRecommendations.to_csv("/user-home/libraries/Sampled_data/datasets/HybridResult.csv", index=False)

### Hybrid Recommender can also be used to predict similar items given some item IDs

In [ ]:
hybrid_model.predict_similar_items(item_features=item_indicator_features,item_ids=[3,55,90], n_similar=10)